In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from itertools import combinations
from scipy import stats
import numpy as np
from statsmodels.stats.power import TTestIndPower
from glob import glob

In [2]:
def raw_score(x, mu_sigmas):
    mu, sigma = mu_sigmas[x['WorkerId']]
    return mu + sigma * x['score']


def get_mu_sigmas(lp):
    mu_sigmas = {}
    worker, mu, sigma = '', 0, 0
    for i, line in enumerate(open('./data/ad-%s-trk-mean-sd.txt' % lp, 'rt')):
        if i % 3 == 0:
            worker = line.split('"')[1]
        if i % 3 == 1:
            mu = float(line.split(' ')[1])
        if i % 3 == 2:
            try:
                sigma = float(line.split(' ')[1])
            except:
                sigma = 0
                print('here')
            mu_sigmas[worker] = (mu, sigma)
    return mu_sigmas

In [3]:
all_df = None

for fn in glob('./data/ad-*-good-stnd-redup.csv'):
    print(fn)
    lp = fn.split('-')[1]
    df = pd.read_csv(fn, sep='\t')
    mu_sigmas = get_mu_sigmas(lp)
    df['raw_score'] = df.apply(lambda x, mu_sigmas=mu_sigmas: raw_score(x, mu_sigmas), axis=1)
    
    if all_df is None:
        all_df = df
    else:
        all_df = pd.concat([all_df, df])

df = all_df

./data/ad-csen-good-stnd-redup.csv
./data/ad-zhen-good-stnd-redup.csv
here
./data/ad-ruen-good-stnd-redup.csv
./data/ad-haen-good-stnd-redup.csv
./data/ad-isen-good-stnd-redup.csv
here
./data/ad-jaen-good-stnd-redup.csv
./data/ad-deen-good-stnd-redup.csv
here


In [4]:
df = df[df.type.isin(['SYSTEM', 'REPEAT'])]
df.head()

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,raw_score
0,3IKDQS3DQE0Z1Q7G4898UEI39MOICD,M0013,cs,en,ad,4,Facebook-AI.7,1000039,SYSTEM,177,0.620848,931.0,100.0
1,3IKDQS3DQE0Z1Q7G4898UEI39MOICD,M0013,cs,en,ad,4,Facebook-AI.7,1000039,SYSTEM,178,-0.025028,931.0,75.0
2,3IKDQS3DQE0Z1Q7G4898UEI39MOICD,M0013,cs,en,ad,4,Facebook-AI.7,1000039,SYSTEM,179,-0.696739,931.0,49.0
3,3IKDQS3DQE0Z1Q7G4898UEI39MOICD,M0013,cs,en,ad,4,Facebook-AI.7,1000039,SYSTEM,180,-1.678470,931.0,11.0
4,3IKDQS3DQE0Z1Q7G4898UEI39MOICD,M0013,cs,en,ad,4,Facebook-AI.7,1000039,SYSTEM,181,0.620848,931.0,100.0


# Variances

In [5]:
def pooled_variance(groups, variable='score'):
    stds = []
    for i, group in groups:
        if len(group) < 2:
            continue   
        stds.append((len(group), group[variable].std()))
        
    num = 0
    denom = 0

    for n, sigma in stds:
        denom += (n-1)
        num += (n-1) * sigma * sigma

    pooled_var = num / denom
    return pooled_var

In [6]:
# total variance
total_var = df['raw_score'].var()
total_var

866.2223087404275

In [7]:
l = []
for i, g in df.groupby(['sys_id', 'sid', 'rid']):
    l.append(len(g))
all(i == 1 for i in l)

False

In [8]:
# pooled variance
pooled_var = pooled_variance(df.groupby(['sys_id', 'sid', 'rid']), variable='raw_score')
pooled_var

665.9271807914765

In [9]:
true_score_var = total_var - pooled_var
true_score_var

200.29512794895095

In [10]:
print(np.sqrt(total_var))
print(np.sqrt(pooled_var))
print(np.sqrt(true_score_var))

29.431654876007695
25.805564919053342
14.15256612593458


In [11]:
gamma = pooled_var / true_score_var
gamma

3.3247298005231603

In [13]:
true_score_var / total_var

0.23122831855970066